In [3]:
using Interact
using Mux
using Plots
using DelimitedFiles

┌ Info: Precompiling Mux [a975b10e-0019-58db-a62f-e48ff68538c9]
└ @ Base loading.jl:1278
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
│   exception = ErrorException("Required dependency GR [28b8d3ca-fb5f-59d9-8090-bfdbd6d07a71] failed to load from a cache file.")
└ @ Base loading.jl:1042


### read files

In [4]:
Ns=1:10
νs=(1/60)*(0.2:0.2:10.0)
r=readdlm("data/r_squared_delay.txt")
s=readdlm("data/s_squared_delay.txt")
t=readdlm("data/t_squared_delay.txt");

In [5]:
for i in 1:size(r)[1], j in 1:size(r)[2] 
    r[i,j]=min(r[i,j],1)
end

In [6]:
max_r=zeros(length(Ns))
i=0
for N in Ns
    i+=1
    max_r[i]=60*νs[findfirst(x->x>0.05,r[i,:])]
end
colors=[:green, :blue, :purple, :red];

# Δp app

In [7]:
ui= @manipulate for buscost in range(0,0.2,length=101), t_0 in range(0.1,5.0,length=101),
ρ_0 in range(0.1,2.5,length=101), γ in range(0.0,1.0,length=101),colorlog in [-2,-1,0]
    
    #calculate popularity
    I=(((s.+buscost*Ns)./ρ_0).^(γ)) .* ((t./t_0).^(1-γ))
    Δp=map(x->(1-x)/(1+x),I)
    Δp=Δp.*(1 .- r) .+ (-1)*r
    
    #plot popularity
    heatmap(60*νs,Ns,Δp,xlab="requests per minute",ylab="number of buses",title="popularity Δp",yticks=Ns,
            aspectratio=.6,c=[:red,:green],clim=(-10.0^colorlog,10.0^colorlog),frame=:box,size=[900,600],left_margin=8Plots.mm)
    
    #find and plot roots
    for i in 1:length(Ns), j in 1:length(νs)-1
        #stable fixed point
        Δp[i,j]*Δp[i,j+1]>0 ? nothing : scatter!(60*0.5*[νs[j]+νs[j+1]],[Ns[i]],markercolor=(Δp[i,j]>0 ? :white : :black),markersize=5,label=:none)
    end
    
    #plot rejection curve
    plot!(max_r,Ns,linewidth=2,linecolor=:orange,markersize=5,markercolor=:orange,marker=:circle,label=">5% rejected")
end;

In [8]:
@layout! ui vbox(vbox(hbox(:buscost, :t_0, :ρ_0), hbox(:γ, :colorlog)), observe(_));

In [9]:
WebIO.webio_serve(page("/", req -> ui), 8101);